In [1]:
# Set to have less messages from distributed
import dask
dask.config.set({"logging.distributed": "error"})
dask.config.set(fuse_ave_width=50)

In [2]:
# Setup cluster
from dask.distributed import Client, progress
from dask_kubernetes import KubeCluster
cluster = KubeCluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
client

/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://10.32.10.8:37043 Dashboard: /user/0000-0001-6632-0187/proxy/46407/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
# Load some useful modules 
import numpy as np
import xarray as xr
import xrft
import intake

from matplotlib import pyplot as plt
%matplotlib inline

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['geojson', 'postgis', 'shapefile', 'spatialite'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [4]:
cat_url = "https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml"
cat = intake.Catalog(cat_url)

In [5]:
coords = cat.LLC4320_grid(chunks=False).to_dask().isel(face=1).reset_coords()

In [6]:
coords = coords.chunk({'time':1})

In [17]:
sss = cat.LLC4320_SSS().to_dask().isel(face=1, 
                                i = slice(0,2000,10), j = slice(0,2000,10))

In [18]:
sss.SSS.data

,Array,Chunk
Bytes,1.44 GB,160.00 kB
Shape,"(9030, 200, 200)","(1, 200, 200)"
Count,126421 Tasks,9030 Chunks
Type,float32,numpy.ndarray


In [9]:
#sss = sss.SSS.chunk({'time': 24*120, 'i':200, 'j':200})

In [10]:
sss.data

dask.array<rechunk-merge, shape=(9030, 200, 200), dtype=float32, chunksize=(2880, 200, 200), chunktype=numpy.ndarray>

In [11]:
subset_factor = 20
sss_sub = (sss.SSS.isel(j=slice(0 ,None, subset_factor),
                              i=slice(0 ,None, subset_factor), 
                              time=slice(0,120*24*3))
                 .stack(point=['i','j'])
                 .chunk({'time': 24*120, 'point':100}))

In [12]:
sss_sub.data

dask.array<reshape, shape=(8640, 100), dtype=float32, chunksize=(2880, 100), chunktype=numpy.ndarray>

In [13]:
SSS_ps_t = xrft.power_spectrum(sss_sub, dim=['time'],
                           window=True, detrend='constant',
                           chunks_to_segments=True)

In [14]:
SSS_ps_t.data

dask.array<truediv, shape=(3, 2880, 100), dtype=float64, chunksize=(1, 2880, 100), chunktype=numpy.ndarray>

In [15]:
%time SSS_ps_t.load()

KilledWorker: ("('zarr-getitem-15bcbf60288d55f7007270940c127253', 4358, 0, 0)", <Worker 'tcp://10.32.28.4:44875', memory: 0, processing: 119>)

In [16]:
SSS_ps_t.mean(['time_segment','point']).plot()
plt.xscale('log')
plt.yscale('log')

KilledWorker: ("('zarr-getitem-15bcbf60288d55f7007270940c127253', 5821, 0, 0)", <Worker 'tcp://10.32.9.6:43749', memory: 0, processing: 506>)